<a href="https://colab.research.google.com/github/hasankere/Week5_Challenge_project/blob/main/ethiomart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from transformers import get_scheduler
import torch
from torch.utils.data import DataLoader, Dataset
from seqeval.metrics import classification_report
import sqlite3


In [ ]:
from google.colab import files

# Upload file
uploaded = files.upload()



Saving channels_to_crawls.csv to channels_to_crawls.csv


In [ ]:
import pandas as pd

# Load the uploaded file
channels_df = pd.read_csv('channels_to_crawls.csv', header=None)

# Use the first column as the channel list
channel_list = channels_df[0].tolist()
print(channel_list)


['@ZemenExpress', '@nevacomputer', '@meneshayeofficial', '@ethio_brand_collection', '@Leyueqa', '@sinayelj', '@Shewabrand', '@helloomarketethiopia', '@modernshoppingcenter', '@qnashcom', '@Fashiontera', '@kuruwear', '@gebeyaadama', '@MerttEka', '@forfreemarket', '@classybrands', '@marakibrand', '@aradabrand2', '@marakisat2', '@belaclassic', '@AwasMart', '@qnashcom', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [ ]:
import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops


In [ ]:
import time
import asyncio

async def fetch_with_retry(channel_info, limit=100, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            return await fetch_messages(channel_info, limit)
        except OperationalError as e:
            print(f"Database locked, retrying... Attempt {attempt + 1}/{retries}")
            await asyncio.sleep(5)  # Wait 5 seconds before retrying
            attempt += 1
    raise Exception("Max retries reached, database still locked.")


In [ ]:
import asyncio
import time
from telethon.errors import SessionPasswordNeededError
from sqlite3 import OperationalError

async def fetch_with_retry(channel_info, limit=100, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            return await fetch_messages(channel_info, limit)  # Replace with your message-fetching function
        except OperationalError as e:
            print(f"Database is locked, retrying... Attempt {attempt + 1}/{retries}")
            await asyncio.sleep(5)  # Wait 5 seconds before retrying
            attempt += 1
    raise Exception("Max retries reached, database still locked.")


In [ ]:
import pandas as pd
from telethon import TelegramClient
import nest_asyncio
import asyncio
from google.colab import files
import time

# Allow nested event loops for Google Colab
nest_asyncio.apply()

# Telegram API credentials
api_id = '27904749'
api_hash = '5d9f2efaa785a2c4d04b5043088c2813'

# Load the uploaded channels CSV
channels_df = pd.read_csv('channels_to_crawls.csv', header=None)
channel_list = channels_df[0].tolist()  # Use the first column as the channel list

# Initialize Telegram Client
client = TelegramClient(':memory:', api_id, api_hash)

# Retry logic for handling operational errors
def retry_on_lock(func, retries=3, delay=5):
    for _ in range(retries):
        try:
            return func()
        except Exception as e:
            print(f"Error: {e}, retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception("Retries exhausted.")

# Function to fetch messages from a single channel
async def fetch_messages(channel_username, limit=100):
    messages = []
    try:
        async for message in client.iter_messages(channel_username, limit=limit):
            media_path = None
            if message.media:
                media_path = await message.download_media(file='media/')

            messages.append({
                'channel_username': channel_username,
                'id': message.id,
                'message': message.text,
                'date': message.date,
                'media_path': media_path
            })
    except Exception as e:
        print(f"Error fetching messages from {channel_username}: {e}")
    return messages

# Fetch messages from all channels
async def fetch_from_all_channels():
    all_data = []
    for channel_username in channel_list:
        print(f"Fetching messages from {channel_username}...")
        channel_data = await fetch_messages(channel_username)
        all_data.extend(channel_data)
    return all_data

# Main coroutine
async def main():
    retry_on_lock(lambda: asyncio.run(client.start()))
    async with client:
        all_data = await fetch_from_all_channels()

    # Save data to CSV
    df = pd.DataFrame(all_data)
    output_path = '/content/raw_telegram_data.csv'
    df.to_csv(output_path, index=False)
    files.download(output_path)

    print(df.head())

# Run the main coroutine
asyncio.run(main())


Please enter your phone (or bot token): +251912026082
Please enter the code you received: 88642
Signed in successfully as Hasan Kere Obse; remember to not break the ToS or you will risk an account ban!
Fetching messages from @ZemenExpress...
Fetching messages from @nevacomputer...
Fetching messages from @meneshayeofficial...


Fetching messages from @ethio_brand_collection...
Fetching messages from @Leyueqa...
Fetching messages from @sinayelj...
Fetching messages from @Shewabrand...
Fetching messages from @helloomarketethiopia...
Fetching messages from @modernshoppingcenter...
Fetching messages from @qnashcom...
Fetching messages from @Fashiontera...
Fetching messages from @kuruwear...
Fetching messages from @gebeyaadama...
Fetching messages from @MerttEka...
Fetching messages from @forfreemarket...
Fetching messages from @classybrands...
Fetching messages from @marakibrand...
Fetching messages from @aradabrand2...
Fetching messages from @marakisat2...
Fetching messages from @belaclassic...
Fetching messages from @AwasMart...
Fetching messages from @qnashcom...
Fetching messages from nan...
Error fetching messages from nan: Cannot cast float to any kind of Peer.
Fetching messages from nan...
Error fetching messages from nan: Cannot cast float to any kind of Peer.
Fetching messages from nan...
Error fetching 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                      
1    @ZemenExpress  6023                                                      
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                       date                               media_path  
0 2025-01-17 05:06:37+00:00                               media/.jpg  
1 2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2 2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3 2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4 2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [1]:
import nltk
nltk.download('punkt')  # Download the punkt tokenizer data
nltk.download('stopwords')  # Download stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# Import libraries
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK resources
nltk.download('punkt')  # Download the punkt tokenizer
nltk.download('stopwords')  # Download stopwords

# Step 1: Load the Dataset
from google.colab import files

# Upload your dataset file
uploaded = files.upload()

# Load the dataset into a DataFrame
dataset = pd.read_csv('raw_telegram_data.csv')

# Display the first few rows
print("Original Dataset:")
print(dataset.head())




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving raw_telegram_data.csv to raw_telegram_data (3).csv
Original Dataset:
  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [23]:
# Step 2: Define Preprocessing Functions
# Function to clean text
def clean_text(text):
    """
    Cleans the given text by removing non-Amharic characters, digits, and extra spaces.
    """
    if pd.isnull(text):  # Handle missing values
        return ''
    # Remove non-Amharic characters and keep spaces
    text = re.sub(r'[^\u1200-\u137F\s]', '', str(text))
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Normalize multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Tokenize text
def tokenize_text(text):
    """
    Tokenizes the cleaned text into words using NLTK's word_tokenize.
    """
    if not text:  # Handle empty or null text gracefully
        return []
    return word_tokenize(text)

# Remove stopwords
def remove_stopwords(tokens):
    """
    Removes stopwords from the list of tokens.
    """
    # Define a set of Amharic stopwords for efficient lookup
    amharic_stopwords = set(['ነው', 'እና', 'እርስዎ', 'እንዲሁም', 'በኩል', 'የሚሆነው'])
    return [word for word in tokens if word not in amharic_stopwords]


In [17]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
import pandas as pd

# Dummy function to demonstrate how to label entities (replace this with your actual labeling logic)
def label_in_conll_format(text):
    """
    Function to label tokens (words) in the text. Replace this function with the logic for NER labeling.
    The function returns the tokens in CoNLL format with predefined entity labels.
    """
    # Tokenize the text (simple example, you may need a more sophisticated tokenizer)
    tokens = text.split()

    labeled_text = []

    for token in tokens:
        # Example: Assign labels based on certain conditions (replace with your actual logic)

        # English entity recognition
        if token.lower() in ['product', 'item']:  # Example condition for English product
            label = 'PRODUCT'
        elif token.lower() in ['etb', 'usd']:  # Example: label currencies like ETB, USD as PRICE
            label = 'PRICE'
        elif token.lower() in ['addis', 'ethiopia']:  # Example: label cities/countries as LOCATION
            label = 'LOCATION'

        # Amharic entity recognition (example words, modify as needed)
        elif token in ['እቃ', 'አምስት']:  # 'እቃ' is Amharic for 'product', 'አምስት' is 'five'
            label = 'PRODUCT'
        elif token in ['ብር', 'ኢትቢ'] :  # Amharic currencies (like 'ብር' for Birr, 'ኢትቢ' for ETB)
            label = 'PRICE'
        elif token in ['አዲስ', 'ኢትዮጵያ']:  # 'አዲስ' for Addis Ababa and 'ኢትዮጵያ' for Ethiopia
            label = 'LOCATION'
        else:
            label = 'O'  # O stands for "Other", no entity

        labeled_text.append(f"{token} {label}")

    # Return the labeled text in CoNLL format (each token + label on its own line)
    return "\n".join(labeled_text)

def save_to_conll(df, output_path):
    """
    Function to save labeled data in CoNLL format (each token is labeled in a separate line).
    """
    with open(output_path, 'w') as f:
        for index, row in df.iterrows():
            # Ensure that 'processed_content' column is not empty or NaN before labeling
            if isinstance(row['processed_content'], str) and row['processed_content'].strip():
                labeled_text = label_in_conll_format(row['processed_content'])
                f.write(labeled_text)
                f.write("\n")  # Add an empty line after each sentence

# Assuming you have already preprocessed the data and stored it in 'df'
# Here's an example dataframe (including both English and Amharic)
df = pd.DataFrame({
    'sender': [12345, 67890],
    'timestamp': ['2025-01-16 10:00', '2025-01-16 10:05'],
    'processed_content': ['This is a product costing 100 ETB in Addis Ababa.',
                          'እቃ ሲሆን በ አዲስ ኢትዮጵያ 200 ብር ይዘዋል።']  # Example Amharic message
})

# Save the labeled data to a CoNLL file
output_path = '/content/labeled_telegram_data.conll'  # Save to Colab's environment
save_to_conll(df, output_path)

# Display the file path where the labeled data is saved
print(f"Labeled data saved to {output_path}")


Labeled data saved to /content/labeled_telegram_data.conll


In [25]:
import pandas as pd

# Function to label tokens in CoNLL format (with B- and I- for entities)
def label_in_conll_format(text):
    """
    Function to label tokens (words) in the text. Replace this function with the logic for NER labeling.
    The function returns the tokens in CoNLL format with predefined entity labels.
    """
    tokens = text.split()
    labeled_text = []

    # Variables to track entity labeling (for B- and I- tags)
    current_entity = None  # Keeps track of the current entity
    for i, token in enumerate(tokens):
        # Product Entity Labeling
        if token.lower() in ['baby', 'bottle', 'laptop', 'phone', 'chair']:  # Example products
            if current_entity != 'PRODUCT':
                label = 'B-PRODUCT'  # Start of a new product entity
                current_entity = 'PRODUCT'
            else:
                label = 'I-PRODUCT'  # Inside the product entity
        # Location Entity Labeling
        elif token.lower() in ['addis', 'ababa', 'bole', 'ethiopia']:  # Example locations
            if current_entity != 'LOC':
                label = 'B-LOC'  # Start of a new location entity
                current_entity = 'LOC'
            else:
                label = 'I-LOC'  # Inside the location entity
        # Price Entity Labeling
        elif token.isdigit() or token in ['ብር', 'etb']:  # Example for price (ETB, digits)
            if current_entity != 'PRICE':
                label = 'B-PRICE'  # Start of a new price entity
                current_entity = 'PRICE'
            else:
                label = 'I-PRICE'  # Inside the price entity
        else:
            label = 'O'  # Outside any entity (e.g., common words or punctuation)
            current_entity = None  # Reset entity if no match

        labeled_text.append(f"{token} {label}")

    # Return the labeled text in CoNLL format (each token + label on its own line)
    return "\n".join(labeled_text)

def save_to_conll(df, output_path):
    """
    Function to save labeled data in CoNLL format (each token is labeled in a separate line).
    """
    with open(output_path, 'w') as f:
        for index, row in df.iterrows():
            # Ensure that 'processed_content' column is not empty or NaN before labeling
            if isinstance(row['processed_content'], str) and row['processed_content'].strip():
                labeled_text = label_in_conll_format(row['processed_content'])
                f.write(labeled_text)
                f.write("\n\n")  # Add an empty line after each sentence

# Assuming you have already preprocessed the data and stored it in 'df'
# Here's an example dataframe (including both English and Amharic)
df = pd.DataFrame({
    'sender': [12345, 67890],
    'timestamp': ['2025-01-16 10:00', '2025-01-16 10:05'],
    'processed_content': [
        'This is a baby bottle costing 100 ETB in Addis Ababa.',
        'እቃ ሲሆን በ አዲስ ኢትዮጵያ 200 ብር ይዘዋል።'
    ]  # Example messages in both English and Amharic
})

# Save the labeled data to a CoNLL file
output_path = '/content/labeled_telegram_data.conll'  # Save to Colab's environment
save_to_conll(df, output_path)

# Display the file path where the labeled data is saved
print(f"Labeled data saved to {output_path}")


Labeled data saved to /content/labeled_telegram_data.conll


In [28]:
# Step 4: Separate Metadata and Message Content
# Check column names to ensure correct extraction
if 'channel_username' not in dataset.columns:
    print("Error: 'channel_username' column not found. Please check the column name.")
else:
    structured_data = dataset[['channel_username', 'id', 'date', 'cleaned_message', 'tokens_no_stopwords']]

    # Step 5: Store the preprocessed data in a structured format (CSV)
    output_file = 'structured_data.csv'
    structured_data.to_csv(output_file, index=False)

    # Display the structured data for review
    print("Structured Data Preview:")
    print(structured_data.head())

    # Download the structured data CSV file
    files.download(output_file)


Structured Data Preview:
  channel_username    id                       date  \
0    @ZemenExpress  6024  2025-01-17 05:06:37+00:00   
1    @ZemenExpress  6023  2025-01-17 05:06:37+00:00   
2    @ZemenExpress  6022  2025-01-17 05:06:37+00:00   
3    @ZemenExpress  6021  2025-01-17 05:06:27+00:00   
4    @ZemenExpress  6020  2025-01-16 12:49:25+00:00   

                                     cleaned_message tokens_no_stopwords  
0                                                                     []  
1                                                                     []  
2  የህፃናት መመገቢያ ጡጦ ዋጋ፦ ብር ውስን ፍሬ ነው ያለው አድራሻ ቁመገናኛ...                  []  
3  የህፃናት መመገቢያ ጡጦ ዋጋ፦ ብር ውስን ፍሬ ነው ያለው አድራሻ ቁመገናኛ...                  []  
4  የጡት ጫፍ ዋጋ፦ ብር አድራሻ ቁመገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢ...                  []  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
print(dataset.columns)


Index(['channel_username', 'id', 'message', 'date', 'media_path',
       'cleaned_message', 'tokens', 'tokens_no_stopwords'],
      dtype='object')


In [34]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize

# Download the correct punkt tokenizer and all languages support
nltk.download('punkt')
nltk.download('punkt_tab')  # This downloads the tab-specific tokenizer

# Load your dataset (assuming it's already loaded in `dataset`)
dataset = pd.read_csv("raw_telegram_data.csv")

# Extract a subset of the dataset for labeling
subset = dataset['message'].head(10)  # Adjust the number of rows based on the subset you need

# Example of entity labels (expand these as needed)
product_keywords = ['አፕል', 'ስልኩ', 'መኪና']  # Example product-related keywords in Amharic
location_keywords = ['ኢትዮጵያ', 'አዲስ', 'ተማ']  # Example location-related keywords in Amharic

# Function to label the entities
def label_entities(text):
    if pd.isnull(text):  # Check for NaN or None values
        return ""

    tokens = word_tokenize(str(text))  # Convert to string in case it's float or other type
    labeled_tokens = []

    for token in tokens:
        label = 'O'  # Default label is 'O' (for outside any entity)

        # Check for product-related words
        if any(keyword in token for keyword in product_keywords):
            label = 'B-PRODUCT'
        # Check for location-related words
        elif any(keyword in token for keyword in location_keywords):
            label = 'B-LOCATION'

        # Add the token and its label to the list
        labeled_tokens.append(f"{token} {label}")

    return '\n'.join(labeled_tokens)

# Apply the labeling function to the subset of messages
labeled_data = subset.apply(label_entities)

# Format the CoNLL output: Each message gets separated by a blank line
conll_output = '\n\n'.join(labeled_data)

# Write to a file
with open("labeled_data.conll", 'w', encoding='utf-8') as f:
    f.write(conll_output)

# Print out the labeled data preview
print("CoNLL Labeled Data Preview:")
print(conll_output)


CoNLL Labeled Data Preview:




💥💥 O
................................... O
💥💥 O
🎯 O
Baby O
Feeding O
Bottle O
Spoon O
with O
Container O
Baby O
Food O
Feeder O
💯 O
High O
Quality O
🍼የህፃናት O
መመገቢያ O
ጡጦ O
👍Silicone O
Squeeze O
Spoon O
Feeder O
for O
Infant O
Food O
Dispensing O
and O
Feeding O
👍Safe O
Material O
👍Filterable O
Food O
👍Strong O
Sealing O
👍Easy O
to O
Use O
ዋጋ፦ O
💵🏷 O
400 O
ብር O
♦️ውስን O
ፍሬ O
ነው O
ያለው O
🔥🔥🔥 O
🏢 O
አድራሻ👉 O
📍ቁ.1️⃣♦️መገናኛ O
መሰረት O
ደፋር O
ሞል O
ሁለተኛ O
ፎቅ O
ቢሮ O
ቁ. O
S05/S06 O
📍 O
ቁ.2️⃣♦️ፒያሳ O
ጊዮርጊስ O
አደባባይ O
ራመት_ታቦር_ኦዳ_ህንፃ O
1ኛ O
ፎቅ O
ሱቅ O
ቁ. O
G1 O
-107 O
💧💧💧💧 O
📲 O
0902660722 O
📲 O
0928460606 O
➡️ O
🔴ፒያሳ O
ቅርንጫፍ O
0941337070 O
🔖 O
💬በTelegram O
ለማዘዝ O
⤵️ O
ይጠቀሙ🔽 O
@ O
zemencallcenter O
@ O
zemenexpressadmin O
ለተጨማሪ O
ማብራሪያ O
የቴሌግራም O
ገፃችን⤵️ O
https O
: O
//telegram.me/zemenexpress O
💥💥 O
................................... O
💥💥 O

💥💥 O
................................... O
💥💥 O
🎯 O
Baby O
Feeding O
Bottle O
Spoon O
with O
Container O
Baby O
Food O
Feeder O
💯 O
High

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [35]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')  # To ensure compatibility for tokenization

# Example keywords for Product, Location, and Price in Amharic (expand this list based on your needs)
product_keywords = ['ስልኩ', 'አፕል', 'መኪና', 'ቡና', 'በረም']
location_keywords = ['አዲስ', 'ተማ', 'ኢትዮጵያ', 'ቦሌ', 'ሐምሌ']
price_keywords = ['ዋጋ', 'ብር', 'እትም', 'በ', '100']

# Load dataset
dataset = pd.read_csv("raw_telegram_data.csv")

# Extract a subset of the dataset for labeling (30-50 messages)
subset = dataset['message'].head(50)  # Adjust the number of rows based on the subset size

# Function to label the entities (Product, Location, Price)
def label_entities(text):
    if pd.isnull(text):  # Check for NaN values
        return ""

    tokens = word_tokenize(str(text))  # Ensure the text is in string format
    labeled_tokens = []

    prev_label = None  # To handle the continuity of the entity

    for token in tokens:
        label = 'O'  # Default label is 'O' (outside any entity)

        # Check for product-related words
        if any(keyword in token for keyword in product_keywords):
            if prev_label != 'B-PRODUCT':
                label = 'B-PRODUCT'  # Beginning of a product entity
            else:
                label = 'I-PRODUCT'  # Inside a product entity
            prev_label = 'B-PRODUCT' if label == 'B-PRODUCT' else 'I-PRODUCT'

        # Check for location-related words
        elif any(keyword in token for keyword in location_keywords):
            if prev_label != 'B-LOC':
                label = 'B-LOC'  # Beginning of a location entity
            else:
                label = 'I-LOC'  # Inside a location entity
            prev_label = 'B-LOC' if label == 'B-LOC' else 'I-LOC'

        # Check for price-related words
        elif any(keyword in token for keyword in price_keywords):
            if prev_label != 'B-PRICE':
                label = 'B-PRICE'  # Beginning of a price entity
            else:
                label = 'I-PRICE'  # Inside a price entity
            prev_label = 'B-PRICE' if label == 'B-PRICE' else 'I-PRICE'

        # Add the token and its label to the list
        labeled_tokens.append(f"{token} {label}")

    return '\n'.join(labeled_tokens)

# Apply the labeling function to the subset of messages
labeled_data = subset.apply(label_entities)

# Format the CoNLL output: Each message gets separated by a blank line
conll_output = '\n\n'.join(labeled_data)

# Write the labeled data to a CoNLL format file
with open("labeled_data.conll", 'w', encoding='utf-8') as f:
    f.write(conll_output)

# Print out the labeled data preview
print("CoNLL Labeled Data Preview:")
print(conll_output)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


CoNLL Labeled Data Preview:




💥💥 O
................................... O
💥💥 O
🎯 O
Baby O
Feeding O
Bottle O
Spoon O
with O
Container O
Baby O
Food O
Feeder O
💯 O
High O
Quality O
🍼የህፃናት O
መመገቢያ O
ጡጦ O
👍Silicone O
Squeeze O
Spoon O
Feeder O
for O
Infant O
Food O
Dispensing O
and O
Feeding O
👍Safe O
Material O
👍Filterable O
Food O
👍Strong O
Sealing O
👍Easy O
to O
Use O
ዋጋ፦ B-PRICE
💵🏷 O
400 O
ብር I-PRICE
♦️ውስን O
ፍሬ O
ነው O
ያለው O
🔥🔥🔥 O
🏢 O
አድራሻ👉 O
📍ቁ.1️⃣♦️መገናኛ O
መሰረት O
ደፋር O
ሞል O
ሁለተኛ O
ፎቅ O
ቢሮ O
ቁ. O
S05/S06 O
📍 O
ቁ.2️⃣♦️ፒያሳ O
ጊዮርጊስ O
አደባባይ O
ራመት_ታቦር_ኦዳ_ህንፃ O
1ኛ O
ፎቅ O
ሱቅ O
ቁ. O
G1 O
-107 O
💧💧💧💧 O
📲 O
0902660722 O
📲 O
0928460606 O
➡️ O
🔴ፒያሳ O
ቅርንጫፍ O
0941337070 O
🔖 O
💬በTelegram B-PRICE
ለማዘዝ O
⤵️ O
ይጠቀሙ🔽 O
@ O
zemencallcenter O
@ O
zemenexpressadmin O
ለተጨማሪ O
ማብራሪያ O
የቴሌግራም O
ገፃችን⤵️ O
https O
: O
//telegram.me/zemenexpress O
💥💥 O
................................... O
💥💥 O

💥💥 O
................................... O
💥💥 O
🎯 O
Baby O
Feeding O
Bottle O
Spoon O
with O
Container O
Baby O
Food O

In [48]:
from google.colab import files
uploaded = files.upload()


Saving raw_telegram_data.csv to raw_telegram_data (4).csv


In [49]:
import pandas as pd
from datasets import Dataset

# Load the uploaded CSV file
df = pd.read_csv("raw_telegram_data.csv")

# Convert the pandas DataFrame into a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Check the dataset structure
print(dataset)


Dataset({
    features: ['channel_username', 'id', 'message', 'date', 'media_path'],
    num_rows: 2200
})


In [51]:
import pandas as pd

# Convert Hugging Face dataset to pandas DataFrame
df = pd.DataFrame(dataset)

# Check if 'message' column exists
print(df.columns)

# Now apply the preprocess_text function to the 'message' column
df['cleaned_message'] = df['message'].apply(preprocess_text)


Index(['channel_username', 'id', 'message', 'date', 'media_path'], dtype='object')


In [56]:
import pandas as pd
from datasets import Dataset
import re
import nltk
from transformers import XLMRobertaTokenizer

# Download required resources
nltk.download("punkt")

# Assuming the dataset is uploaded, let's load it into a pandas DataFrame
df = pd.read_csv("raw_telegram_data.csv")

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Define text preprocessing functions
def preprocess_text(text):
    """Preprocess text: remove unwanted characters, tokenize, etc."""
    text = re.sub(r'[^\u1200-\u137F\s]', '', str(text))  # Keep only Amharic characters
    return text

# Preprocess the 'message' column
df['cleaned_message'] = df['message'].apply(preprocess_text)

# Now proceed with labeling and fine-tuning...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
import pandas as pd
from datasets import Dataset
import re
import nltk
from transformers import XLMRobertaTokenizer

# Download required resources for NLTK
nltk.download("punkt")

# Assuming the dataset is uploaded, let's load it into a pandas DataFrame
df = pd.read_csv("raw_telegram_data.csv")  # Replace with your dataset path

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Define text preprocessing functions
def preprocess_text(text):
    """Preprocess text: remove unwanted characters, tokenize, etc."""
    text = re.sub(r'[^\u1200-\u137F\s]', '', str(text))  # Keep only Amharic characters
    return text

# Convert the Hugging Face Dataset back to pandas DataFrame for easier manipulation
df = pd.DataFrame(dataset)

# Preprocess the 'message' column
df['cleaned_message'] = df['message'].apply(preprocess_text)

# Now proceed with labeling and fine-tuning...
print(df.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                               None   
1    @ZemenExpress  6023                                               None   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  \
0  2025-01-17 05:06:37+00:00                               media/.jpg   
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg   
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg   
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4   
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg   

                                     cleaned_message  
0                              

In [71]:
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import evaluate  # Use evaluate for metrics
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cpu


In [75]:
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizer

# Load the pretrained XLM-Roberta model for token classification (NER task)
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base")

# Load the tokenizer for XLM-Roberta
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [77]:
import pandas as pd

# Load the raw telegram dataset
df = pd.read_csv('raw_telegram_data.csv')

# Preview the dataset
df.head()


,channel_username,id,message,date,media_path
0,@ZemenExpress,6024,NaN,2025-01-17 05:06:37+00:00,media/.jpg
1,@ZemenExpress,6023,NaN,2025-01-17 05:06:37+00:00,media/photo_2025-01-17_05-06-37.jpg
2,@ZemenExpress,6022,💥💥...................................💥💥\n\n🎯 B...,2025-01-17 05:06:37+00:00,media/photo_2025-01-17_05-06-37 (1).jpg
3,@ZemenExpress,6021,💥💥...................................💥💥\n\n🎯 B...,2025-01-17 05:06:27+00:00,media/document_2025-01-17_05-06-27.mp4
4,@ZemenExpress,6020,💥💥...................................💥💥\n\n📌 R...,2025-01-16 12:49:25+00:00,media/photo_2025-01-16_12-49-25.jpg


In [80]:
df['message'] = df['message'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)



In [83]:
# Replace NaN values with an empty string or a placeholder
df['message'] = df['message'].fillna('UNKNOWN')

# If you prefer a placeholder, you can use:
# df['message'] = df['message'].fillna('UNKNOWN')


In [87]:
# Drop rows where 'message' column contains NaN values
df = df.dropna(subset=['message'])

# Verify the result
print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                            UNKNOWN   
1    @ZemenExpress  6023                                            UNKNOWN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [88]:
import pandas as pd

# Load the dataset
df = pd.read_csv('raw_telegram_data.csv')

# Check the first few rows of the dataset
print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [91]:
# Replace NaN values in 'message' column with empty strings
df['message'] = df['message'].fillna('')

# Alternatively, you can drop rows with NaN in the 'message' column:
# df = df.dropna(subset=['message'])

# Check the cleaned data
print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                      
1    @ZemenExpress  6023                                                      
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [93]:
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification

# Load the pre-trained XLM-Roberta model for token classification (NER)
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base')


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
import pandas as pd

# Load your dataset (replace 'raw_telegram_data.csv' with the actual file path)
df = pd.read_csv('raw_telegram_data.csv')

# Check the first few rows to understand the structure of your dataset
print(df.head())


  channel_username    id                                            message  \
0    @ZemenExpress  6024                                                NaN   
1    @ZemenExpress  6023                                                NaN   
2    @ZemenExpress  6022  💥💥...................................💥💥\n\n🎯 B...   
3    @ZemenExpress  6021  💥💥...................................💥💥\n\n🎯 B...   
4    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   

                        date                               media_path  
0  2025-01-17 05:06:37+00:00                               media/.jpg  
1  2025-01-17 05:06:37+00:00      media/photo_2025-01-17_05-06-37.jpg  
2  2025-01-17 05:06:37+00:00  media/photo_2025-01-17_05-06-37 (1).jpg  
3  2025-01-17 05:06:27+00:00   media/document_2025-01-17_05-06-27.mp4  
4  2025-01-16 12:49:25+00:00      media/photo_2025-01-16_12-49-25.jpg  


In [102]:
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification

# Load the pre-trained XLM-Roberta model for token classification (NER)
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base')

# Example: Tokenizing and preparing data
text = "ስም የእቃ ስም እባክህ ምን ይኸው የእቃ ተመን?"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
print(inputs)

# You can fine-tune the model as shown earlier using the Hugging Face Trainer.




Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[     0,  37851,  30179,   6550,  37851,   4708,   5657,   4133,   4359,
          14719, 220555,  30179,   6550, 131253,    548,     32,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [104]:
import pandas as pd
from datasets import Dataset
from transformers import XLMRobertaTokenizer

# Load your labeled data (in CoNLL format)
def load_conll(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.read().strip().split('\n\n')

    sentences = []
    labels = []

    for line in lines:
        sentence = []
        label = []
        for word in line.splitlines():
            parts = word.split()
            if len(parts) == 2:
                sentence.append(parts[0])
                label.append(parts[1])
        sentences.append(sentence)
        labels.append(label)

    return sentences, labels

# Load your dataset
sentences, labels = load_conll('labeled_data.conll')

# Create a DataFrame
df = pd.DataFrame({
    'tokens': sentences,
    'labels': labels
})

# Convert DataFrame to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)


In [106]:
# Inspect the first example in the dataset
print(dataset[0])  # This will print the first record (tokens and labels)


{'tokens': ['💥💥', '...................................', '💥💥', '🎯', 'Baby', 'Feeding', 'Bottle', 'Spoon', 'with', 'Container', 'Baby', 'Food', 'Feeder', '💯', 'High', 'Quality', '🍼የህፃናት', 'መመገቢያ', 'ጡጦ', '👍Silicone', 'Squeeze', 'Spoon', 'Feeder', 'for', 'Infant', 'Food', 'Dispensing', 'and', 'Feeding', '👍Safe', 'Material', '👍Filterable', 'Food', '👍Strong', 'Sealing', '👍Easy', 'to', 'Use', 'ዋጋ፦', '💵🏷', '400', 'ብር', '♦️ውስን', 'ፍሬ', 'ነው', 'ያለው', '🔥🔥🔥', '🏢', 'አድራሻ👉', '📍ቁ.1️⃣♦️መገናኛ', 'መሰረት', 'ደፋር', 'ሞል', 'ሁለተኛ', 'ፎቅ', 'ቢሮ', 'ቁ.', 'S05/S06', '📍', 'ቁ.2️⃣♦️ፒያሳ', 'ጊዮርጊስ', 'አደባባይ', 'ራመት_ታቦር_ኦዳ_ህንፃ', '1ኛ', 'ፎቅ', 'ሱቅ', 'ቁ.', 'G1', '-107', '💧💧💧💧', '📲', '0902660722', '📲', '0928460606', '➡️', '🔴ፒያሳ', 'ቅርንጫፍ', '0941337070', '🔖', '💬በTelegram', 'ለማዘዝ', '⤵️', 'ይጠቀሙ🔽', '@', 'zemencallcenter', '@', 'zemenexpressadmin', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን⤵️', 'https', ':', '//telegram.me/zemenexpress', '💥💥', '...................................', '💥💥'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [107]:
def tokenize_and_align_labels(examples):
    # Ensure that sentences are properly tokenized
    tokenized_inputs = tokenizer(examples['tokens'], padding=True, truncation=True, is_split_into_words=True)

    # Ensure labels align with the tokenized sentences
    labels = examples['labels']
    new_labels = []
    for i, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=0)):
        if word_ids is None:
            new_labels.append(-100)  # Special tokens (like [CLS], [SEP]) are ignored
        else:
            new_labels.append(labels[word_ids])

    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs


In [108]:
# Remove empty sentences or rows that don't have tokens or labels
dataset = dataset.filter(lambda example: len(example['tokens']) > 0 and len(example['labels']) > 0)


Filter:   0%|          | 0/48 [00:00<?, ? examples/s]

In [112]:
from transformers import XLMRobertaTokenizerFast

# Load the fast tokenizer for XLM-Roberta
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')


In [115]:
import evaluate

# Replace load_metric with evaluate.load
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    true_predictions = [
        [model.config.id2label[p] for p in prediction if p != -100]
        for prediction in predictions
    ]
    true_labels = [
        [model.config.id2label[l] for l in label if l != -100]
        for label in labels
    ]
    return metric.compute(predictions=true_predictions, references=true_labels)
